### Imports 

In [1]:
from acessos import read, get_conn, persistir_uma_linha, persistir_multiplas_linhas, replace_df, update_banco
import pandas as pd
from math import*
from tqdm.auto import tqdm  # for notebooks
from connections.mysql_connector import MySQL_Connector
import sys

In [1]:
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)


x = [1,2,3,4,5,6,7,8,9,10]
y = [1,1,1,1,1,1,1,1,1,1]
jaccard_similarity(x,y)

0.1

In [3]:
def calcular_scores(df_avaliador_1, df_avaliador_2, avaliador_1, avaliador_2, verbose=False):
    
    #Refatorar esta linha, pois pode dar erro em dataFrames com tamnanhos diferentes!!
    lista_sentencas = list(df_avaliador_1['sentenca_id'].unique())

    df_scores = pd.DataFrame()

    for sentenca_id in tqdm(lista_sentencas): 
        if verbose == True:
            print("Sentenca ID {}".format(sentenca_id))


        df_sentencas_av1 = df_avaliador_1.loc[df_avaliador_1['sentenca_id'] == sentenca_id]
        df_sentencas_av2 = df_avaliador_2.loc[df_avaliador_2['sentenca_id'] == sentenca_id]


        lista_grupos_avaliador_1 = df_sentencas_av1["num_grupo"].unique()
        lista_grupos_avaliador_2 = df_sentencas_av2["num_grupo"].unique()

#         print("  -> Total grupos Av1: {}".format(len(lista_grupos_avaliador_1)))
#         print("  -> Total grupos Av2: {}".format(len(lista_grupos_avaliador_2)))


        for i in range(0,len(lista_grupos_avaliador_1)+1):
    
            lista_tuplas_grupo_av1 = list(df_sentencas_av1.loc[df_sentencas_av1['num_grupo'] == i]['tupla_id'])
            
            
            for j in range(0,len(lista_grupos_avaliador_2)+1):
                score = 0  
               
                lista_tuplas_grupo_av2 = list(df_sentencas_av2.loc[df_sentencas_av2['num_grupo'] == j]['tupla_id'])
                
                
            
                #iterando tuplas do grupo metodo 1
#                 print(lista_tuplas_grupo_av1)
                for t in lista_tuplas_grupo_av1:
                    if t in lista_tuplas_grupo_av2:
                        score=jaccard_similarity(lista_tuplas_grupo_av1, lista_tuplas_grupo_av2)

                    else: 
                        continue

                linha_dataFrame = {"sentenca_id": sentenca_id,
                                  "avaliador1": avaliador_1,
                                  "avaliador2": avaliador_2,
                                  "grupo_av1": i,
                                  "grupo_av2": j,
                                  "score": score
                                  }
                df_scores = df_scores.append(linha_dataFrame, ignore_index=True)
                if verbose == True:
                    print("    -> calculando av1 grupo {} com av2 grupo {}".format(i,j))
                    print(" {}  contra  {}  ".format(lista_tuplas_grupo_av1,lista_tuplas_grupo_av2))
                    print("        -> Score {}".format(score))
    return df_scores

def get_df_tuplas(tag_metodo):
    query = '''
                SELECT * FROM tupla_grupo tg 
                    INNER JOIN grupo_resumo gr USING (grupo_id) 
                    INNER JOIN espaco_amostral_sentenca using(sentenca_id)
                WHERE tag_metodo = '{}'
            '''.format(tag_metodo)
    
    return read(conn, query)
    
def get_tag_metodos():
    query =  "SELECT distinct tag_metodo FROM grupo_resumo;"
    return read(conn, query)

def mostrar_boas_vindas():
    df_tag_metodos = get_tag_metodos()
    lista_tag_metodos = list(df_tag_metodos['tag_metodo'])
    print("Olá! Favor inserir os parâmetros de execução (avaliador1 e avaliador2):")
    print("Selcione os avaliadores a partir dos seguintes tag_metodos:\n")
    print(lista_tag_metodos )
    print("\nInsira o avaliador_1:")
    avaliador_1 = input() 
    print("Insira o avaliador_2:")
    avaliador_2 = input()
    return avaliador_1, avaliador_2
    

def alerta_dataframe_diferente():
    print("**********")
    print("**** Atenção os Métodos possuem tamanhos diferentes!! ****")
    print("Deseja continuar? (sim/nao)")
    continuar = input()
    if continuar == "sim":
        return True
    else:
        return False
    
def buscar_similaridade_grupos(avaliador1, avaliador2, sentenca_id): 
    query =  '''SELECT * FROM avaliacao_similaridade_grupos 
                where avaliador1 ='{}' and avaliador2 ='{}' and sentenca_id={} ;'''.format(avaliador1, avaliador2,sentenca_id)
    return read(conn, query)

def calcular_media(avaliador1, avaliador2, sentenca_id):
    df_medias = buscar_similaridade_grupos(avaliador1, avaliador2,sentenca_id)
    media = df_medias['score'].mean()
    return media

### Calculando Similaridade

In [133]:
connector = MySQL_Connector("conn_orfeu")
db_name = "flairs2"
conn = connector.return_conn(db_name)


# avaliador_1 = "padrao_ouro_mcla"
# avaliador_2 = "padrao_ouro_hbgf"

avaliador_1, avaliador_2 = mostrar_boas_vindas()

print(" --> Carregando Av1")
df_avaliador_1 = get_df_tuplas(avaliador_1)
print(" --> Carregando Av2")
df_avaliador_2 = get_df_tuplas(avaliador_2)

print("   --> Tamanho Av1: {}".format(df_avaliador_1.shape[0]))
print("   --> Tamanho Av2: {}".format(df_avaliador_2.shape[0]))

if df_avaliador_1.shape[0] != df_avaliador_2.shape[0]:
    continuar = alerta_dataframe_diferente()
    if not continuar:
        sys.exit()


        
# df_avaliador_1 = df_avaliador_1.loc[df_avaliador_1["sentenca_id"]== 201]
# df_avaliador_2 = df_avaliador_2.loc[df_avaliador_2["sentenca_id"]== 201]
        
df_score = calcular_scores(df_avaliador_1, df_avaliador_2, avaliador_1, avaliador_2, verbose=False)

replace_df(df_score, "avaliacao_similaridade_grupos",conn)

lista_sentencas = list(df_avaliador_1['sentenca_id'].unique())

for sentenca_id in lista_sentencas:
    score = calcular_media(avaliador_1, avaliador_2, sentenca_id)
    
    dict_df = {
        "sentenca_id": [sentenca_id],
        "avaliador1": [avaliador_1],
        "avaliador2": [avaliador_2],
        "score": score
    }
    df_media = pd.DataFrame(dict_df)
    replace_df(df_media, "avaliacao_similaridade_medias",conn)

Olá! Favor inserir os parâmetros de execução (avaliador1 e avaliador2):
Selcione os avaliadores a partir dos seguintes tag_metodos:

['word2vec-70-OLD', ' distancia-cosseno-80-OLD', 'bert-cosseno-70-OLD', 'bert-cosseno-80-OLD', 'word2vec-80-OLD', 'distancia-cosseno-70-OLD', 'tsne_distortion', 'tsne_distortion_grupo_100', 'padrao_ouro_hbgf', 'padrao_ouro_mcla', 'word2vec-70', 'word2vec-80', 'bert-cosseno-70', 'bert-cosseno-80', 'distancia-cosseno-80', 'distancia-cosseno-70']

Insira o avaliador_1:


 padrao_ouro_hbgf


Insira o avaliador_2:


 distancia-cosseno-70


 --> Carregando Av1
 --> Carregando Av2
   --> Tamanho Av1: 4146
   --> Tamanho Av2: 4146



Iniciando Persistencia
Sucesso na inserção


1

### Persistindo Coeficientes de Concordância

In [14]:
connector = MySQL_Connector("conn_orfeu")
db_name = "flairs2"
conn = connector.return_conn(db_name)

dict_data = {
    "padrao_ouro": ["All", "All", "Nmf", "Nmf", "Cspa", "Cspa", "Hgpa", "Hgpa", "Mcla", "Mcla", "Hgbf", "Hgbf", "Entre Avaliadores", "Entre Avaliadores"],
    "algoritmo": ["ICC", "Krippendorff", "ICC", "Krippendorff", "ICC", "Krippendorff", "ICC", "Krippendorff", "ICC", "Krippendorff","ICC", "Krippendorff", "ICC", "Krippendorff"],
    "coerencia": [0.873, 0.7513, 0.877, 0.760, 0.896, 0.7863, 0.893, 0.7911, 0.873, 0.7516, 0.898, 0.7917, 0.87, 0.75]
}

df_stats = pd.DataFrame(dict_data)

replace_df(df_stats, "testes_estatisticos",conn)

df_stats.head()

,padrao_ouro,algoritmo,coerencia
0,All,ICC,0.8730
1,All,Krippendorff,0.7513
2,Nmf,ICC,0.8770
3,Nmf,Krippendorff,0.7600
4,Cspa,ICC,0.8960


### Montando dataset de testes estatísticos:
#### - KRUSKAL-WALLIS 
#### - NEMENYI


In [55]:
connector = MySQL_Connector("conn_orfeu")
db_name = "flairs2"
conn = connector.return_conn(db_name)

query = '''

SELECT avaliador1, avaliador2, total_grupo_av1, total_grupo_av2, grupos_iguais, grupos_iguais/total_grupo_av1 as perc
    FROM
    (
        SELECT avaliador1, avaliador2, SUM(total_grupo_av1) total_grupo_av1, SUM(total_grupo_av2) total_grupo_av2
            FROM
            (
                SELECT 
                        avaliador1, 
                        avaliador2, 
                        sentenca_id, 
                        MAX(grupo_av1) total_grupo_av1,
                        MAX(grupo_av2) total_grupo_av2 
                        FROM 
                            avaliacao_similaridade_grupos
                        GROUP BY 
                            1,2,3
            ) grupo_sent
            GROUP BY 1,2
    ) total_grupos
    INNER JOIN
    (
        SELECT  
                avaliador1,
                avaliador2, 
                SUM(case when score >= {} then 1 else 0 end) grupos_iguais 
                FROM 
                    avaliacao_similaridade_grupos
                GROUP BY
                    1,2
    ) match_grupo
    USING (avaliador1, avaliador2)
WHERE avaliador1 = "padrao_ouro_mcla"
GROUP BY avaliador1, avaliador2
ORDER BY grupos_iguais desc
'''

df_perc  = pd.DataFrame(columns = ['avaliador1', 'avaliador2', 'total_grupo_av1', 'total_grupo_av2', 'grupos_iguais', 'perc', 'limiar'])

for i in tqdm(range(5,105,5)):
    limiar = i/100
    
    df_apoio = read(conn, query.format(limiar))
    df_apoio['limiar'] = limiar
    df_perc = df_perc.append(df_apoio, ignore_index=True)
    
#df_perc = df_perc[['avaliador2', 'perc', 'limiar']].transpose()
df_perc.to_csv(r'base_teste_estatistico_sem_transpor.csv', sep=";", encoding='utf-8')